#   _Key Derivation Functions_ - Funkcje wyprowadzania klucza
## LBK, Kollbi (Insert names later)
###   Podstawowe informacje
1.  Funkcje KDF to algorytmy kryptograficzne zwracające sekret lub wiele sekretów z tajnej wartości.    
        Np. Stworzenie klucza prywatnego (jeden sekret) z hasła użytkownika (tajna wartość).
1.  Metody te powstały ponieważ hasła użytkowników są przewidywalne i krótkie (niska entropia). Pojawia się potrzeba stworzenia mocnego klucza.  
Jednak nikt nie będzie pamiętał 64 znakowego hasła. Dlatego z pomocą przychodzi `key-stretching`.  
Proces w którym tworzony jest bezpieczniejszy i (teoretycznie) odporniejszy na ataki siłowe nowy klucz.  
`Key-streching` jest jedną z kluczowych technik zwiększania bezpieczeństwa klucza. Do key-streching'u można zaliczyć proste funkcje hashujące jak SHA, MD5 lub szyfry blokowe.
1. Funkcje wyprowadzania klucza powinny być `deterministyczne`. Oznacza to, że dla tych samych danych wejściowych otrzymamy zawsze ten sam skrót niezależnie ile razy funkcja zostanie wywołana.
1. Key Derivation Functions mogą być używane nie tylko do zapisywania haseł w postaci skrótów w bazie danych. Można je wykorzystywać między innymi do symetrycznego szyfrowania dysków lub kryptografi asymetrycznej.
```
                                               _             _  ______  _____                                   _       
     _ __   __ _ ___ _____      _____  _ __ __| |           | |/ /  _ \|  ___|           ___  ___  ___ _ __ ___| |_ ___  
    | '_ \ / _` / __/ __\ \ /\ / / _ \| '__/ _` |   =====\  | ' /| | | | |_     ====\   / __|/ _ \/ __| '__/ _ \ __/ __|
    | |_) | (_| \__ \__ \\ V  V / (_) | | | (_| |   =====/  | . \| |_| |  _|    ====/   \__ \  __/ (__| | |  __/ |_\__ \
    | .__/ \__,_|___/___/ \_/\_/ \___/|_|  \__,_|           |_|\_\____/|_|              |___/\___|\___|_|  \___|\__|___/
    |_|                                                     
```

### Funkcje KDF to nie tylko funkcje haszujące! Jednak mają wspólną część
1. Tak samo jak funkcje haszujące szyfrują dane deterministycznie. (To samo wejście - to samo wyjście)
1. Na wyjściu jest zawsze ciąg o takiej samej długości. Bez znaczenia jak duże były dane wejściowe.
1. Działa tylko w jedną stronę. Nie odzyska wejścia danych z wyjścia.


### Zrozumienie od podstaw

Najprostszą funkcję wyprowadzenia klucza można stworzyć wykorzystując dowolny algorytm haszujący. Dla przykładu użyty zostanie algorytm SHA256.  

function(`password`, `sha256`) --> `key`

In [5]:
from hashlib import sha256

hash = sha256(b"password")

print(hash.digest().hex())

5e884898da28047151d0e56f8dc6292773603d0d6aabbdd62a11ef721d1542d8


Należy pamiętać, że powyższe rozwiązanie nie jest bezpieczne. Proste hasze są podatne na ataki słownikowe (np. tęczowe tablice (Rainbow table)). Jak zatem zwiększyć bezpieczeństwo?

Pierwszym krokiem jest dodanie `soli` (ang. `salt`).  
`Salt` jest wartością przechowywaną razem z uzyskanym kluczem. Używana, żeby utrudnić atak siłowy np. Rainbow Table. Sprawia, że nowy hasz jest bardziej skomplikowany i zmniejsza szansę pojawienia się go we wcześniej obliczonych tęczowych tablicach.

function_with_salt(`salt`, `password`, `sha256`) --> `key`  

In [6]:
password = b"password"
salt = b"salt"
hash = sha256(salt+password)

print(hash.digest().hex())

13601bda4ea78e55a07b98866d2be6be0744e3866f13c00c811cab608a28f322


Funkcję wyżej można opisać jako `HKDF` (`HMAC-based key derivation function`) - najprostsza funkcja wyprowadzania klucza.  
Ciągle jest mniej bezpieczna niż aktualne KDF. Zaleca się korzystanie z PBKDF2, Bcrypt, Scrypt (ten projekt) oraz Argon2.



### Przykład 1.0
Wykorzystanie `HMAC` (Hash-based message authentication code) z solą, hasłem użytkownika oraz funkcją SHA256 do pozyskania klucza.
1. Sprawdź co się stanie jak zmienisz sól. Może to być bardzo mała zmiana. Porównaj parę wyników dla tego samego hasła. 
1. Czy klucze z nową solą znacznie różnią się od głównego? Jeśli tak to dlaczego. ([Lawinowość](https://en.wikipedia.org/wiki/Avalanche_effect))
1. Porównaj dwa klucze z tym samym hasłem w następujący sposób:  
    1. Dla samej funkcji haszującej `sha256(b"password")`
    1. Następnie stwórz dwie różne sole.
    1. Porównaj klucze dołączając sól po haśle `sha256(b"password+salt")`

In [7]:
from hmac import HMAC
from hashlib import sha256

salt = b'Crypto'
password = b'IsFun'

key = HMAC(salt, password, digestmod=sha256)

print(key.digest().hex())  # Desired output: `a773d6dd2e141ddc22a578716a5da01e695286e03abf3b09abb3c48c296e1a70`

a773d6dd2e141ddc22a578716a5da01e695286e03abf3b09abb3c48c296e1a70


### Zasosowanie `soli`
Wielokrotne użycie soli dla tego samego hasła umożliwia pozyskanie wielu innych kluczy. Jest to często stosowane rozwiązanie.  
Dla przykładu: można posiadać w bazie dwóch użytkowników z takim samym hasłem. Zwykłe zahaszowanie hasła w obu przypadkach zwróci to samo.  
Jednak wygenerowanie dla każdego z nich unikalnej wartości (`salt`) i dołączenie do hasła, sprawia że otrzymane hasze są diametralnie różne. ([Lawinowość](https://en.wikipedia.org/wiki/Avalanche_effect))  
KDF pomimo znajomości klucza z `salt1` nie jest wstanie znaleźć klucza z `salt2`.

### Po podstawach

Wiedząc co rozróżnia funkcje haszujące od KDF oraz czym jest sól, zwrócona zostanie uwaga na to, co powinna zapewniać dobra funkcja wyprowadzania klucza.  

1. Memory Hardness - cecha funkcji zapewniająca, że wymaga dużej ilości pamięci do obliczeń lub wymaga znacznie większego nakładu czasowego, tak by maszyny wykonujące obliczenia równoległe nie były znacząco bardziej efektywne w obliczeniach od standardowych procesorów CPU.  
Algorytm określa się "memory-hard" gdy potrzeba wysokiej ilości pamięci do pojedynczego wykonania. 

1. Funkcja KDF, która ma być wysoce odporna na ataki siłowe lub słownikowe, powinna maksymalnie spowolnić możliwość obliczeń równoległych. Jednocześnie wykonując relatywnie szybkie obliczenie klucza bądź skrótu.

1. Żeby "opóźnić" generację kluczy można zwiększyć złożoność obliczeniową, złożoność pamięciową lub oba jednocześnie.

Więcej informacji: `Funkcje wyprowadzania klucza (KDF) wykorzystujące odwzorowanie logistyczne - Grzegorz Frejek`